In [1]:
import pandas as pd
import src.cleaning_functions as cf
import re
import numpy as np

I load my csv file that contains the raw data.

In [2]:
df = pd.read_csv("data/attacks.csv",encoding = "ISO-8859-1")

I explore 3 random rows in the dataframe.

In [3]:
df.sample(3)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
23382,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12613,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2096,1999.10.01,01-Oct-1999,1999.0,Unprovoked,USA,Hawaii,Old Kona Airport State Park,"Surfing, lying on surfboard",Jesse Spencer,M,...,"Tiger shark, 1.8 m to 2.4 m [6' to 8']",Star Bulletin,1999.10.01-Spencer.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1999.10.01,1999.10.01,4207.0,NaN,NaN


I use my function "prepare" (see cf.prepare.__doc__) in the columns to ease the data analysis.
Then I delete the rows where I only have Nan.
I explore again 5 random rows to see the changes and I explore the number of Nan in my dataframe and the number of rows and columns using .shape mehtod.

In [4]:
df.columns = cf.prepare(df.columns)
df.dropna(how="all", inplace=True)
df.head(5)
df.isna().sum()
df.shape

(8703, 24)

During data exploration, I realised the column "unnamed : 22" has a single entry that is not NaN, so I explore it. The entry is "stopped here", which is not useful, so I decide to delete the column.

In [5]:
unnamed22NaN=df["unnamed:_22"].unique()
unnamed22NaN
df.drop(["unnamed:_22"], axis=1, inplace=True)

I also realised the column "unnamed : 23" has only 2 entries that are not NaN, so I explore them.
The entries are 'Teramo' and 'change filename', which are not useful, so I decide to delete the column.

In [6]:
unnamed23NaN=df["unnamed:_23"].unique()
unnamed23NaN
df.drop(["unnamed:_23"], axis=1, inplace=True)

I save the columns in a variable  that includes links to webs in case I had to use them and delete the columns from the dataframe

In [7]:
pdfcol = df["href_formula"],df["href"]
df.drop(["href_formula","href"],axis=1, inplace=True)

Now that I have explored more about my data, I decide that my hypothesis will be:

Are surfers the most targeted by sharks?

Do sharks attack more often in Australia on summer?

In order to prove my hypothesis, I delete the columns that are not relevant for my study. Then, I delete the rows where there is only NaN.

In [8]:
df.drop(["case_number", "year", "type", "area", "location", "name", "sex","age", "injury", "time","species", "investigator_or_source","pdf", "case_number.1","case_number.2", "original_order"],axis=1, inplace=True)
df.dropna(how="all", inplace=True)

In order to work with the column "fatal_(y/n)" I use my function "strdata" (see cf.prepare.__doc__)to cast the data into string so I can use my function "prepare" (see cf.prepare.__doc__) to ease the data analysis. Then, I use my function fatal (see cf.fatal.__doc__) to unify the registers into the categories "yes", "no", or Nan.

In [9]:
df["fatal_(y/n)"] = cf.strdata(df["fatal_(y/n)"])
df["fatal_(y/n)"] = cf.prepare(df["fatal_(y/n)"])

df["fatal_(y/n)"] = (df["fatal_(y/n)"]).apply(cf.fatal)

I will use my funtion  "strdata" to convert to string the data in the column "country" and be able to use "prepare" function in "activity" column.

In [10]:
df.activity = cf.strdata(df.activity)
df.activity = cf.prepare(df.activity)

I  will  use my function "unify" to classify the data that tells me the same information into a single string so I can count its occurrence. Every register in the "activity" column that contains the words "surf" will become "surfing".
I will repeat the same process for every column.

In [11]:
#I unify in the category "surf"
list_surfing = ["surf"]
for i in list_surfing:
    df.activity = cf.unify(df.activity,i,"surfing")

#I unify in the category "swimming"
list_swimming = ["swim", "diving", "bath", "bathing", "dive","scuba", "snorkel"]
for i in list_swimming:
    df.activity = cf.unify(df.activity,i,"swimming")

#I unify in the category "fishing"
list_fishing = ["fish", "line"]
for i in list_fishing:
    df.activity = cf.unify(df.activity,i,"fishing")

#I unify in the category "boat"
list_boat = ["boat", "yacht", "ship","sail"]
for i in list_boat:
    df.activity = cf.unify(df.activity,i,"boat")

#I unify in the category "walking on the water"
list_walkonwater = ["wad", "stand", "walk"]
for i in list_walkonwater:
    df.activity = cf.unify(df.activity,i,"walking on the water")

#I unify in the category "taking photographs or filming"
list_photo = ["photo", "picture","film", "record"]
for i in list_photo:
    df.activity = cf.unify(df.activity,i,"taking photographs or filming")

I will use my function "act" to unify my data and add "other_activities" where the activities are a minority and not as repeated.

In [12]:
df.activity = df.activity.apply(cf.act)

I will use my funtion  "strdata" to convert to string the data in the column "country" and be able to use "prepare" function.

In [13]:
df.country = cf.strdata(df.country)
df.country = cf.prepare(df.country)
df.head()

,date,country,activity,fatal_(y/n)
0,25-Jun-2018,usa,other_activities,no
1,18-Jun-2018,usa,other_activities,no
2,09-Jun-2018,usa,surfing,no
3,08-Jun-2018,australia,surfing,no
4,04-Jun-2018,mexico,swimming,no


I will use my funtion  "strdata" to convert to string the data in the column "date" and be able to use "prepare" function.

Then, I use my "getmonth" function to get create a new column including only the name of the months. After that, I apply my function "australia" to create a new column where only "australia" or Nan will appear.

In [14]:
df.date = cf.strdata(df.date)
df.date = cf.prepare(df.date)

df["month"] = df.date.apply(cf.getmonth)
df["australia"] = df.country.apply(cf.australia)
df.head()

,date,country,activity,fatal_(y/n),month,australia
0,25-jun-2018,usa,other_activities,no,june,NaN
1,18-jun-2018,usa,other_activities,no,june,NaN
2,09-jun-2018,usa,surfing,no,june,NaN
3,08-jun-2018,australia,surfing,no,june,australia
4,04-jun-2018,mexico,swimming,no,june,NaN


I save "df" dataframe in a csv file.

In [15]:
df.to_csv("graphs_and_tables/df.csv")

I create a copy of the dataframe to work with it and leave the old one as it is. Then, I eliminate the columns "date" and "country" because I don't need them anymore for my studies.

In [16]:
dfaus = df.copy()
dfaus.drop(["date","country"], axis=1, inplace=True)
dfaus.sample(4)

,activity,fatal_(y/n),month,australia
4685,swimming,no,NaN,NaN
5404,swimming,no,march,NaN
1346,other_activities,no,august,NaN
1366,other_activities,no,july,NaN


I will create a new column called "season", where "summer" will appear if the "month" column is December, January or February (these are the "southern" summer months). It will do the same with the other seasons.

In [17]:
dfaus["season"] = dfaus.month.apply(cf.season)
dfaus.head()

,activity,fatal_(y/n),month,australia,season
0,other_activities,no,june,NaN,winter
1,other_activities,no,june,NaN,winter
2,surfing,no,june,NaN,winter
3,surfing,no,june,australia,winter
4,swimming,no,june,NaN,winter


I save dataframe "dfaus" in a csv file.

In [18]:
dfaus.to_csv("graphs_and_tables/dfaus.csv")

I create a copy of the former dataframe called "dfhist". Then I use my function "strdata" to cast my data into string and be able to work with it. After that, I apply my function "actnan" to add Nan where my activities are not met.

In [19]:
dfhist = df.copy()

dfhist["activity"] = cf.strdata(dfhist.activity)
dfhist["activity"] = dfhist.activity.apply(cf.actnan)

I delete the columns that I don't need for my study and I change the NaN values for the word "unknown".

In [20]:
dfhist.drop(["date","country","month","australia"], axis=1, inplace=True)
dfhist["fatal_(y/n)"].fillna("unknown", inplace=True)
dfhist.head()

,activity,fatal_(y/n)
0,NaN,no
1,NaN,no
2,surfing,no
3,surfing,no
4,swimming,no


I save the "dfhist" table in a csv a file.

In [21]:
dfhist.to_csv("graphs_and_tables/dfhist.csv")